In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import json
import random
import numpy as np

import torch 
from torch import nn 
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
import random
import copy
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
import cv2
import json

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE",
                        "#FFDD00",
                        "#FF7D00",
                        "#FF006D",
                        "#ADFF02",
                        "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
model_ident = "base_dataset"
unique_iden = "visualize"

main_dir = "/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Devin/SkeletonAE"
data_dir = os.path.join(main_dir,"kinetics-skeleton")
train_dir = os.path.join(data_dir,"kinetics_train")
train_info = os.path.join(data_dir,"kinetics_train_label.json")
val_dir = os.path.join(data_dir,"kinetics_val")
val_info = os.path.join(data_dir,"kinetics_val_label.json")


models_saves = os.path.join(main_dir,"model_saves")
embeddings_save = os.path.join(main_dir,"embedding_save")
test_vids = os.path.join(main_dir,"test_vids")
class_names = os.listdir(data_dir)
batch_size = 128

#os.makedirs(models_saves,exist_ok=True)
#os.makedirs(embeddings_save,exist_ok=True)

In [8]:
with open(train_info,"r") as f0:
  load_train_info = json.load(f0)

with open(val_info,"r") as f0:
  load_val_info = json.load(f0)

In [9]:
load_train_info["---QUuC4vJs"]

{'has_skeleton': True, 'label': 'testifying', 'label_index': 354}

In [11]:
!du -h --max-depth=1 "/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Devin/SkeletonAE/kinetics-skeleton"

3.2G	/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Devin/SkeletonAE/kinetics-skeleton/kinetics_val
du: cannot read directory '/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Devin/SkeletonAE/kinetics-skeleton/kinetics_train': Input/output error
4.0K	/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Devin/SkeletonAE/kinetics-skeleton/kinetics_train
3.2G	/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Devin/SkeletonAE/kinetics-skeleton


In [ ]:
lis_train = []
for __k in tqdm(os.listdir(train_dir)):
  try:
    if load_train_info[__k]['has_skeleton']:
      lis_train.append(__k)
  except KeyError:
    continue

In [ ]:
lis_val = []
for __k in tqdm(os.listdir(val_dir)):
  try:
    if load_val_info[__k]['has_skeleton']:
      lis_val.append(__k)
  except KeyError:
    continue

In [ ]:
len(lis_train),len(lis_val)

In [ ]:
with open(os.path.join(train_dir,"---QUuC4vJs.json"),"r") as f0:
  test_data = json.load(f0)

In [ ]:
len(test_data["data"][0]["skeleton"][1]["pose"])

In [ ]:
mapping_l = [
        [15, 13], [13, 11], [11, 5],
        [12, 14], [14, 16], [12, 6],
        [3, 1],[1, 2],[1, 0],[0, 2],[2,4],
        [9, 7], [7,5], [5, 6],
        [6, 8], [8, 10],
        ]


def gen_random_video(model, dataset, epoch, batch_size, model_name, unique_name, epoch_vids, device, mapping_l):
    ind = random.randint(0, len(dataset) - 1)
    batch_sel = random.randint(0, batch_size - 1)

    in_seq, tar_seq, action, vid_size = dataset[ind]
    pred_seq, _ = model(in_seq.repeat(batch_size, 1, 1).to(device))

    os.makedirs(f"{epoch_vids}/{model_name}/{unique_name}/{epoch}", exist_ok=True)
    gen_video(pred_seq[batch_sel].squeeze().cpu().detach().numpy(),
              f"{epoch_vids}/{model_name}/{unique_name}/{epoch}/pred.mp4", int(vid_size[0]), int(vid_size[1]),
              mapping_list=mapping_l)
    gen_video(in_seq.detach().numpy(), f"{epoch_vids}/{model_name}/{unique_name}/{epoch}/true.mp4", int(vid_size[0]),
              int(vid_size[1]), mapping_list=mapping_l)


def gen_skeleton(frame, connections, height, width):
    img_3 = np.zeros([height, width, 3], dtype=np.uint8)
    img_3.fill(255)

    font = cv2.FONT_HERSHEY_SIMPLEX

    # org
    org = (50, 50)

    # fontScale
    fontScale = 1

    # Blue color in BGR
    color = (255, 0, 0)

    # Line thickness of 2 px
    thickness = 2

    # add circles
    for i, coord in enumerate(frame):
        x, y = int(width * coord[0]), int(height * coord[1])
        img_3 = cv2.circle(img_3, center=(x, y), radius=3, color=(255, 0, 0), thickness=6)
        img_3 = cv2.putText(img_3, str(i), (x, y + 5), font,
                            fontScale, color, thickness, cv2.LINE_AA)

    for line in connections:
        i, j = line
        st = frame[i, :]
        start_point = (int(width * st[0]), int(height * st[1]))

        en = frame[j, :]
        end_point = (int(width * en[0]), int(height * en[1]))

        img_3 = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3


def gen_skeletons(frame1, frame2, connections, height, width):
    img_3 = np.zeros([height, width, 3], dtype=np.uint8)
    img_3.fill(255)

    # add circles for the frame-set-1
    for coord in frame1:
        x, y = int(width * coord[0]), int(height * coord[1])
        img_3 = cv2.circle(img_3, center=(x, y), radius=1, color=(255, 0, 0), thickness=6)

    # add circles for the frame-set-2
    for coord in frame2:
        x, y = int(width * coord[0]), int(height * coord[1])
        img_3 = cv2.circle(img_3, center=(x, y), radius=1, color=(0, 255, 0), thickness=6)

    for line in connections:
        i, j = line

        st = frame1[i, :]
        start_point = (int(width * st[0]), int(height * st[1]))
        en = frame1[j, :]
        end_point = (int(width * en[0]), int(height * en[1]))
        img3_ = cv2.line(img_3, start_point, end_point, color=(250, 0, 0), thickness=3)

        st = frame2[i, :]
        start_point = (int(width * st[0]), int(height * st[1]))
        en = frame2[j, :]
        end_point = (int(width * en[0]), int(height * en[1]))
        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 250, 0), thickness=3)

    return img_3


def gen_video(points, save_file, frame_h, frame_w, mapping_list=None):
    # make 3D if points are flatten
    if len(points.shape) == 2:
        fts = points.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points = np.transpose(np.array([points[:, x_cds], points[:, y_cds], points[:, z_cds]]), (1, 2, 0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file,
                             cv2.VideoWriter_fourcc(*'MJPG'),
                             10, size)

    for __id, frame in enumerate(points):
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()


def gen_cmp_video(points1, points2, save_file, frame_h, frame_w, mapping_list=None):
    # make 3D if points are flatten
    if len(points1.shape) == 2:
        fts = points1.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points1 = np.transpose(np.array([points1[:, x_cds], points1[:, y_cds], points1[:, z_cds]]), (1, 2, 0))

    if len(points2.shape) == 2:
        fts = points2.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points2 = np.transpose(np.array([points2[:, x_cds], points2[:, y_cds], points2[:, z_cds]]), (1, 2, 0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file,
                             cv2.VideoWriter_fourcc(*'MJPG'),
                             10, size)

    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    if not mapping_list:
        # add lines
        mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10),
                        (9, 11)]
    for __id, frame_1 in enumerate(points1):
        frame_2 = points2[__id]
        skel_image = gen_skeletons(frame_1, frame_2, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()


In [ ]:
def load_json(file_path):
  with open(file_path,"r") as f0:
    return json.load(f0)

{'MryVk2Hk468',
 'JAAFAjh-3QE',
 'LAp-v5WjnXI',
 'ShMp7sX0Dvs',
 'UcvVsFjOO9o',
 'nqCYmxiaEDs',
 'wUlOB7r6lWk',
 '4qSaCvxDdZ0',
 'wB1pr0x1iRU',
 'a1_N0dhyuSA',
 'xVWMkPizGww',
 '9D_O5ynEU9U',
 'Eozn86k9BTk',
 'rzpWRK0v0AM',
 'oKsOFcF3Z0s',
 'pesd2mKmuoY',
 'OtcJMIJTTx4',
 'xnBxmiHWJC4',
 'oGirUlUBwW0',
 'lt9y-tkK0fU',
 'EuQtBbtEncI',
 'oHW2sRo01TM',
 'IPja_DXBAdA',
 'qLsjneX5O00',
 'UNwiz1lSlKk',
 'LaLiyEfAmWg',
 'NHASiTpYRzM',
 'b9p-RcIBBII',
 'inmlyCBobgI',
 'o5pDdmLt4aE',
 'Jtuw5kSeEOY',
 'Hqlheysvu2Y',
 'L2Z_gqdyiM0',
 'BGASPhD7Xjw',
 '-5rZ-3xLUEc',
 'xybWhgiJlmk',
 'PDd5XAXyY7k',
 'sErqGtXNnUA',
 'UxaVGmELb0o',
 '9wb8Nm2gdmM',
 'A0SHJ1e7JUk',
 'i8KXkP6jajY',
 '39KIhdj_Pcs',
 'Aq_SCLk1Y0Q',
 'xE99hRc-LTY',
 'ml2DWNFJ4Pk',
 'RLCOIVqeoqU',
 'uILvuiaB_6o',
 '3OiV0tSuE9g',
 'ToHJo53ipSs',
 'M5eJMS0jgAg',
 'AZZSbBEFTIw',
 'Y8Y1ZYIDW9s',
 'Fl-1W7wfK3k',
 '7iJ_jgq006w',
 'OHs4pHIqHPE',
 '3Z5I-tlLLMQ',
 'PHTAeicQ8As',
 'k9amQZya0iA',
 'Koy7iGinjD0',
 'lHraXnvY-vQ',
 'bYVa4Rdeowc',
 'poUodG

In [ ]:
lis_train[0]

'--3ouPhoy2A'

In [ ]:
load_json(os.path.join(train_dir,lis_train[1]+".json"))

FileNotFoundError: ignored

In [ ]:
def json_to_numpy()